In [1]:

import nengo
import numpy as np
import time

import tkinter as tk
import os
from PIL import Image
from sklearn.model_selection import train_test_split

from nengo_extras.data import one_hot_from_labels
from nengo_extras.matplotlib import tile
from nengo_extras.vision import Gabor, Mask

import threading
import queue
import pickle
import nengo_dl
import tensorflow as tf

C:\Users\User\AppData\Roaming\Python\Python310\site-packages\nengo_dl\version.py:58: UserWarning: This version of NengoDL has not been tested with your Nengo version (4.0.0). The latest fully supported version is 3.2.0.
  warnings.warn(warnstr)


In [3]:
%pip install --user nengo_dl


  Using cached nengo_dl-3.6.0-py3-none-any.whl
  Using cached tensorflow-2.10.1-cp310-cp310-win_amd64.whl (455.9 MB)


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
import random

%matplotlib inline
style.use('fivethirtyeight')
sns.set(style='whitegrid', color_codes=True)

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_curve, roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder

from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam, SGD, Adagrad, Adadelta, RMSprop
from tensorflow.keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard, CSVLogger, ReduceLROnPlateau

import tensorflow as tf
import random as rn

import os
from random import shuffle
from zipfile import ZipFile
from PIL import Image

In [3]:
input_width = 320
input_height = 120 
n_hid = 1000

lookup = dict()
reverselookup = dict()
count = 0
for j in os.listdir('./input/leapgestrecog/leapGestRecog/00/'):
    if not j.startswith('.'): # If running this code locally, this is to 
                              # ensure you aren't reading in hidden folders
        lookup[j] = count
        reverselookup[count] = j
        count = count + 1
lookup


x_data = []
y_data = []
datacount = 0 # We'll use this to tally how many images are in our dataset
for i in range(0, 10): # Loop over the ten top-level folders
    for j in os.listdir('./input/leapgestrecog/leapGestRecog/0' + str(i) + '/'):
        if not j.startswith('.'): # Again avoid hidden folders
            count = 0 # To tally images of a given gesture
            for k in os.listdir('./input/leapgestrecog/leapGestRecog/0' + 
                                str(i) + '/' + j + '/'):
                                # Loop over the images
                img = Image.open('./input/leapgestrecog/leapGestRecog/0' + 
                                 str(i) + '/' + j + '/' + k).convert('L')
                                # Read in and convert to greyscale
                img = img.resize((input_width, input_height))
                arr = np.array(img)
                x_data.append(arr) 
                count = count + 1
            y_values = np.full((count, 1), lookup[j]) 
            y_data.append(y_values)
            datacount = datacount + count
x_data = np.array(x_data, dtype = 'float32')
x_data = x_data / 255 * 2 - 1
print(x_data.shape)
#x_data = x_data.reshape(datacount, -1)

y_data = np.array(y_data)
y_data = y_data.reshape(datacount)


(20000, 120, 320)


In [4]:
y_data=to_categorical(y_data)
x_data = x_data.reshape((datacount, input_height, input_width, 1))

print(x_data.shape)
print(y_data.shape)
x_train,x_further,y_train,y_further = train_test_split(x_data,y_data,test_size = 0.2, random_state=42)
x_validate,x_test,y_validate,y_test = train_test_split(x_further,y_further,test_size = 0.5, random_state=42)

(20000, 120, 320, 1)
(20000, 10)


In [18]:
batch_size = 128

In [28]:
model = Sequential()
model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same',activation ='relu', input_shape = (input_height,input_width,1)))

model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same',activation ='relu'))


model.add(Conv2D(filters =96, kernel_size = (3,3),padding = 'Same',activation ='relu'))


model.add(Conv2D(filters = 96, kernel_size = (3,3),padding = 'Same',activation ='relu'))
model.add(GlobalAveragePooling2D())


model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dense(10))

In [55]:
# input
inp = tf.keras.Input(shape=(input_height, input_width, 1))

# convolutional layers
conv0 = tf.keras.layers.Conv2D(
    filters=32,
    kernel_size=5,
    activation=tf.nn.relu,
)(inp)

avg_pool0 = tf.keras.layers.AveragePooling2D(pool_size=(2, 2))(conv0)

conv1 = tf.keras.layers.Conv2D(
    filters=64,
    kernel_size=3,
    strides=2,
    activation=tf.nn.relu,
)(avg_pool0)
avg_pool1 = tf.keras.layers.AveragePooling2D(pool_size=(2, 2))(conv1)
conv2 = tf.keras.layers.Conv2D(
    filters=64,
    kernel_size=3,
    strides=2,
    activation=tf.nn.relu,
)(avg_pool1)
avg_pool2 = tf.keras.layers.AveragePooling2D(pool_size=(2, 2))(conv2)

# fully connected layer
flatten0 = tf.keras.layers.Flatten()(avg_pool2)
dense0 = tf.keras.layers.Dense(units=512,activation=tf.nn.relu)(flatten0)

dense = tf.keras.layers.Dense(units=10,activation="softmax")(dense0)

model = tf.keras.Model(inputs=inp, outputs=dense)

In [56]:
model.summary()

Model: "model_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_15 (InputLayer)       [(None, 120, 320, 1)]     0         
                                                                 
 conv2d_69 (Conv2D)          (None, 116, 316, 32)      832       
                                                                 
 average_pooling2d_6 (Averag  (None, 58, 158, 32)      0         
 ePooling2D)                                                     
                                                                 
 conv2d_70 (Conv2D)          (None, 28, 78, 64)        18496     
                                                                 
 average_pooling2d_7 (Averag  (None, 14, 39, 64)       0         
 ePooling2D)                                                     
                                                                 
 conv2d_71 (Conv2D)          (None, 6, 19, 64)         369

In [57]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(lr=0.001),
    loss="categorical_crossentropy",
    metrics=["accuracy"])

In [58]:
History = model.fit(x_train, y_train, epochs=7, batch_size=batch_size, verbose=1, validation_data=(x_validate, y_validate))

Epoch 1/7
125/125 [==============================] - 95s 754ms/step - loss: 0.9316 - accuracy: 0.6834 - val_loss: 0.1351 - val_accuracy: 0.9630
Epoch 2/7
125/125 [==============================] - 93s 746ms/step - loss: 0.0557 - accuracy: 0.9866 - val_loss: 0.0240 - val_accuracy: 0.9920
Epoch 3/7
125/125 [==============================] - 94s 754ms/step - loss: 0.0166 - accuracy: 0.9966 - val_loss: 0.0106 - val_accuracy: 0.9975
Epoch 4/7
125/125 [==============================] - 92s 738ms/step - loss: 0.0182 - accuracy: 0.9952 - val_loss: 0.0062 - val_accuracy: 0.9985
Epoch 5/7
125/125 [==============================] - 92s 735ms/step - loss: 0.0066 - accuracy: 0.9983 - val_loss: 0.0060 - val_accuracy: 0.9980
Epoch 6/7
125/125 [==============================] - 92s 735ms/step - loss: 0.0059 - accuracy: 0.9981 - val_loss: 0.0014 - val_accuracy: 0.9995
Epoch 7/7
125/125 [==============================] - 92s 734ms/step - loss: 0.0016 - accuracy: 0.9998 - val_loss: 0.0061 - val_accuracy:

In [60]:
sfr = 20
ndl_model = nengo_dl.Converter(
    model,
    swap_activations={
        tf.nn.relu: nengo.SpikingRectifiedLinear()},
    scale_firing_rates=sfr,
    synapse=0.005,
    inference_only=True)

In [61]:
model.save('my_model_cnn_nengo_pre.h5')

In [87]:
# Tile the test images.
def get_nengo_compatible_test_data_generator(batch_size=100, n_steps=30):
  """
  Returns a test data generator of tiled (i.e. repeated) images.

  Args:
    batch_size <int>: Number of data elements in each batch.
    n_steps <int>: Number of timesteps for which the test data has to
                   be repeated.
  """
  num_images = int(x_test.shape[0]/20)
  # Flatten the images
  reshaped_x_test = x_test.reshape((num_images, 1, -1))
  # Tile/Repeat them for `n_steps` times.
  tiled_x_test = np.tile(reshaped_x_test, (1, n_steps, 1))

  for i in range(0, num_images, batch_size):
    yield (tiled_x_test[i:i+batch_size], y_test[i:i+batch_size])

In [75]:
# Get the probes for Input, first Conv, and the Output layers.
ndl_mdl_inpt = ndl_model.inputs[inp] # Input layer is Layer 0.
ndl_mdl_otpt = ndl_model.outputs[dense] # Output layer is last.
with ndl_model.net:
  nengo_dl.configure_settings(stateful=False) # Optimize simulation speed.
  # Probe for the first Conv layer.
  first_conv_probe = nengo.Probe(ndl_model.layers[conv0])
  # Probe for penultimate dense layer.
  penltmt_dense_probe = nengo.Probe(ndl_model.layers[dense])

In [84]:
print(x_train.shape)

(16000, 120, 320, 1)


In [1]:
n_steps = 30 # Number of timesteps
batch_size = 100
collect_spikes_output = True
ndl_mdl_spikes = [] # To store the spike outputs of the first Conv layer and the
                    # penultimate dense layer whose probes we defined earlier.
ndl_mdl_otpt_cls_probs = [] # To store the true class labels and the temporal
                            # class-probabilities output of the model.

test_batches = get_nengo_compatible_test_data_generator(
    batch_size=batch_size, n_steps = n_steps)

# Run the simulation.
with nengo_dl.Simulator(ndl_model.net, minibatch_size=batch_size) as sim:
  # Predict on each batch.
  for batch in test_batches:
    sim_data = sim.predict_on_batch({ndl_mdl_inpt: batch[0]})
    for y_true, y_pred in zip(batch[1], sim_data[ndl_mdl_otpt]):
      # Note that y_true is an array of shape (10,) and y_pred is a matrix of
      # shape (n_steps, 10) where 10 is the number of classes in CIFAR-10 dataset.
      ndl_mdl_otpt_cls_probs.append((y_true, y_pred))

    # Collect the spikes if required.
    if collect_spikes_output:
      for i in range(batch_size): # Collecting spikes for each image in first batch.
        ndl_mdl_spikes.append({
          first_conv_probe.obj.ensemble.label: sim_data[first_conv_probe][i],
          penltmt_dense_probe.obj.ensemble.label: sim_data[penltmt_dense_probe][i]
        })
      # Not collecting the spikes for rest batches to save memory.
      collect_spikes_output = False

NameError: name 'get_nengo_compatible_test_data_generator' is not defined

In [78]:
print(test_batches.shape)

AttributeError: 'generator' object has no attribute 'shape'

In [ ]:
acc = 0
temporal_cls_probs = [] # To store the temporal class-probabilities of each test image.
for y_true, y_pred in ndl_mdl_otpt_cls_probs:
  # Pick the spiking network's last time-step output, therefore -1 in y_pred.
  temporal_cls_probs.append(y_pred)
  if np.argmax(y_true) == np.argmax(y_pred[-1]):
    acc += 1

print("Spiking network prediction accuracy: %s %%" % (acc * 100/ x_test.shape[0]))

In [ ]:
def plot_probability(ax, true_cls, pred_cls, clss_probs, num_clss=10):
  """
  Plots the temporal variability in predicted class-probabilities.

  Args:
    ax <matplotlib.axes._subplots.AxesSubplot>: Subplot pane.
    true_cls <int>: The true class of the test image.
    pred_cls <int>: The predicted class of the test image from spiking network.
    clss_probs <numpy.ndarray>: The predicted class probabilities at each
                                timestep. Shape: (n_steps, num_clss).
  """
  ax.set_title("True Class: %s, Pred Class: %s" % (true_cls, pred_cls))
  ax.plot(clss_probs)
  ax.legend([str(j) for j in range(num_clss)], loc="upper left")
  ax.set_xlabel("Time in $ms$")
  ax.set_ylabel("Probability")

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(16, 8), facecolor="#00FFFF")

# Plot for the test image at index 0.
plot_probability(
    axs[0],
    np.argmax(y_test[0]),
    np.argmax(temporal_cls_probs[0][-1]), # Last timestep's probability scores.
    temporal_cls_probs[0]
)

# Plot for the test image at index 19.
plot_probability(
    axs[1],
    np.argmax(y_test[19]),
    np.argmax(temporal_cls_probs[19][-1]), # Last timestep's probability scores.
    temporal_cls_probs[19]
)

TEST VERSUCH

In [ ]:
print()

In [70]:
run_network(activation=nengo.RectifiedLinear(), n_steps=10)

(2000, 120, 320, 1)
(400, 120, 320, 10)


ValidationError: input_15 data: should have rank 3 (batch_size, n_steps, dimensions), found rank 4

In [62]:
# Tile the test images.
def get_nengo_compatible_test_data_generator(batch_size=100, n_steps=30):
  """
  Returns a test data generator of tiled (i.e. repeated) images.

  Args:
    batch_size <int>: Number of data elements in each batch.
    n_steps <int>: Number of timesteps for which the test data has to
                   be repeated.
  """
  num_images = x_test.shape[0]
  # Flatten the images
  reshaped_x_test = x_test.reshape((num_images, 1, -1))
  # Tile/Repeat them for `n_steps` times.
  tiled_x_test = np.tile(reshaped_x_test, (1, n_steps, 1))

  for i in range(0, num_images, batch_size):
    yield (tiled_x_test[i:i+batch_size], y_test[i:i+batch_size])

In [ ]:
# Get the probes for Input, first Conv, and the Output layers.
ndl_mdl_inpt = ndl_model.inputs[layer_objs_lst[0]] # Input layer is Layer 0.
ndl_mdl_otpt = ndl_model.outputs[layer_objs_lst[-1]] # Output layer is last.
with ndl_model.net:
  nengo_dl.configure_settings(stateful=False) # Optimize simulation speed.
  # Probe for the first Conv layer.
  first_conv_probe = nengo.Probe(ndl_model.layers[layer_objs_lst[1]])
  # Probe for penultimate dense layer.
  penltmt_dense_probe = nengo.Probe(ndl_model.layers[layer_objs_lst[-2]])

Hier werden noch normale neuronen trainiert


|                     Building network (0%)                    | ETA:  --:--:--
|                     Building network (0%)                    | ETA:  --:--:--
|                     Building network (0%)                    | ETA:  --:--:--
|                     Building network (0%)                    | ETA:  --:--:--
|                     Building network (0%)                    | ETA:  --:--:--
|                     Building network (0%)                    | ETA:  --:--:--
|                     Building network (0%)                    | ETA:  --:--:--
|                     Building network (0%)                    | ETA:  --:--:--
|                     Building network (0%)                    | ETA:  --:--:--
|                     Building network (0%)                    | ETA:  --:--:--
|                     Building network (0%)                    | ETA:  --:--:--
|                     Building network (0%)                    | ETA:  --:--:--
|                     Building network (

ValidationError: input_10 data: should have rank 3 (batch_size, n_steps, dimensions), found rank 4